# Inflation Forecasting

In [2]:
# import library
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
# import data
df = pd.read_csv('./kota-per-bulan.csv')
df

,City,januari_2008,februari_2008,maret_2008,april_2008,mei_2008,juni_2008,juli_2008,agustus_2008,september_2008,...,maret_2022,april_2022,mei_2022,juni_2022,juli_2022,agustus_2022,september_2022,oktober_2022,november_2022,desember_2022
0,KOTA MEULABOH,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.69,1.22,1.26,1.33,1.12,-0.33,0.49,0.19,-0.62,0.98
1,KOTA BANDA ACEH,2.26,1.98,-0.76,-0.80,3.78,2.75,0.25,-0.56,1.67,...,0.59,1.23,0.92,0.76,0.98,-0.32,0.78,-0.49,0.11,0.64
2,KOTA LHOKSEUMAWE,1.89,1.74,1.13,-0.91,0.98,4.38,0.46,1.14,1.29,...,0.85,1.25,1.03,0.45,0.47,-0.98,0.90,0.04,-0.36,1.51
3,KOTA SIBOLGA,2.51,0.46,1.60,-0.62,1.71,2.31,1.27,0.09,1.68,...,0.93,0.38,0.85,1.12,1.07,-0.02,0.33,-1.09,-0.05,1.44
4,KOTA PEMATANG SIANTAR,1.12,0.96,0.96,0.48,1.95,2.88,1.24,0.02,0.12,...,0.77,0.39,0.62,1.36,0.04,-0.79,1.50,-0.54,-0.08,1.61
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,KOTA TERNATE,1.70,1.18,1.76,0.45,2.58,1.17,1.34,1.36,1.55,...,0.34,0.82,0.27,0.33,1.82,-0.24,0.51,-0.49,0.19,1.14
78,MANOKWARI,2.66,1.55,-0.23,0.08,0.42,5.78,4.33,1.79,2.00,...,1.34,1.07,1.54,0.71,0.57,-0.79,-0.64,0.76,0.94,1.85
79,KOTA SORONG,-1.08,-1.89,2.11,1.13,7.18,5.72,4.05,2.01,1.08,...,0.22,1.02,0.61,0.39,1.26,-0.82,1.49,-0.90,0.30,0.01
80,MERAUKE,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,1.86,0.74,0.02,-0.19,0.58,-0.85,0.07,0.12,-0.05,1.33


In [12]:
# change month values to numeric
df['Month'] = df['Month'].replace(['januari'],'1')
df['Month'] = df['Month'].replace(['februari'],'2')
df['Month'] = df['Month'].replace(['maret'],'3')
df['Month'] = df['Month'].replace(['april'],'4')
df['Month'] = df['Month'].replace(['mei'],'5')
df['Month'] = df['Month'].replace(['juni'],'6')
df['Month'] = df['Month'].replace(['juli'],'7')
df['Month'] = df['Month'].replace(['agustus'],'8')
df['Month'] = df['Month'].replace(['september'],'9')
df['Month'] = df['Month'].replace(['oktober'],'10')
df['Month'] = df['Month'].replace(['november'],'11')
df['Month'] = df['Month'].replace(['desember'],'12')
df

,City,Month,Year,Inflation
0,KOTA MEULABOH,1,2008,0.0
1,KOTA BANDA ACEH,1,2008,2.26
2,KOTA LHOKSEUMAWE,1,2008,1.89
3,KOTA SIBOLGA,1,2008,2.51
4,KOTA PEMATANG SIANTAR,1,2008,1.12
...,...,...,...,...
14755,KOTA TERNATE,12,2022,1.14
14756,MANOKWARI,12,2022,1.85
14757,KOTA SORONG,12,2022,0.01
14758,MERAUKE,12,2022,1.33


In [14]:
# make new column based on City value



ValueError: value_name (Inflation) cannot match an element in the DataFrame columns.